In [1]:
# CS 544 Project 2

In [2]:
# import statements
import pandas as pd
import torch

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_trainX = df_train.iloc[:, :-1]
df_trainY = df_train.iloc[:, -1:]
df_testX = df_test.iloc[:, :-1]
df_testY = df_test.iloc[:, -1:]

trainX = torch.tensor(df_trainX.values, dtype = torch.float64)
trainY = torch.tensor(df_trainY.values, dtype = torch.float64)
testX = torch.tensor(df_testX.values, dtype = torch.float64)
testY = torch.tensor(df_testY.values, dtype = torch.float64)

In [4]:
#Q1
trainX.element_size() * trainX.nelement()

83520

In [5]:
#Q2
trainX_float64 = trainX.clone()
trainX_float64 = (trainX_float64.to(torch.float16)).to(torch.float64)
diff_matrix = torch.abs(trainX_float64 - trainX)
float(torch.max(diff_matrix))

0.0

In [6]:
#Q3
torch.cuda.is_available()

False

In [7]:
# Assumptions
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [8]:
#Q4
(testX[0] @ coef).item()

9.844

In [9]:
#Q5
(testX @ coef).sum().item() / testX.shape[0]

12.073632183908048

In [10]:
def f(x):
    return x**2 - 8*x + 19

In [11]:
#Q6
x = torch.tensor(0.0, requires_grad=True)
y = f(x)
float(y)

19.0

In [12]:
#Q7
optimizer = torch.optim.SGD([x], maximize=False, lr=0.01)
for epoch in range(500):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
float(x)

3.999835968017578

In [13]:
coef = torch.zeros((10, 1), dtype = torch.float64, requires_grad=True)

In [14]:
#Q8
(((trainX @ coef) - trainY)**2).sum().item() / trainY.shape[0]

197.8007662835249

In [15]:
#Q9
torch.manual_seed(544)
loss_fn = torch.nn.MSELoss()
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

optimizer = torch.optim.SGD([coef], lr=0.000002)

for epoch in range(500):
    for batch_X, batch_Y in dl:
        predictions = batch_X @ coef
        loss = loss_fn(predictions, batch_Y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

(((trainX @ coef) - trainY)**2).sum().item() / trainY.shape[0]

26.8113940147193

In [16]:
#Q10
(((testX @ coef) - testY)**2).sum().item() / testY.shape[0]

29.05854692548551